# Prepareing notebook

## Installing Fast.ai through clouderizer 

In [0]:
!wget -NS --content-disposition "https://console.clouderizer.com/givemeinitsh/hR1mde3W" && bash ./clouderizer_init.sh

## instaling libraries for plotting images 

In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

#Getting the data 

## Downloading from internet

In [0]:
!apt-get install p7zip-full

In [0]:
mv 'DL#+Beginner.zip' 'DLBeginner.zip'

In [0]:
rm -r DL#+Beginner.zip.1

In [0]:
ls{'/content/'}

In [0]:
!7z x DLBeginner.zip  -o/content/clouderizer/fast.ai/data/

In [0]:
ls{'/content/clouderizer/fast.ai/data/"DL# Beginner"/'}

In [0]:
rm -r /content/DLBeginner.zip

In [0]:
!7z x /content/clouderizer/fast.ai/data/"DL# Beginner"/train.zip -o/content/clouderizer/fast.ai/data/

In [0]:
!7z x /content/clouderizer/fast.ai/data/"DL# Beginner"/test.zip -o/content/clouderizer/fast.ai/data/

In [0]:
!7z x /content/clouderizer/fast.ai/data/"DL# Beginner"/meta-data.zip -o/content/clouderizer/fast.ai/data/

In [0]:
ls{'/content/clouderizer/fast.ai/data/'}

In [0]:
!cp /content/clouderizer/fast.ai/data/meta-data/sample_submission.csv /content/clouderizer/fast.ai/data/
!cp /content/clouderizer/fast.ai/data/meta-data/test.csv /content/clouderizer/fast.ai/data/
!cp /content/clouderizer/fast.ai/data/meta-data/train.csv /content/clouderizer/fast.ai/data/


In [0]:
!rm -r /content/clouderizer/fast.ai/data/"DL# Beginner"/
!rm -r /content/clouderizer/fast.ai/data/meta-data/

In [0]:
ls{'/content/clouderizer/fast.ai/data/'}

In [0]:
!rm -r /content/clouderizer/fast.ai/data/dogscats.zip
!rm -r /content/clouderizer/fast.ai/data/dogscats

In [0]:
ls{'/content/clouderizer/fast.ai/data/'}

# Problems & Fix

## Weights

*   Cannot find  resnext_101_64x4d.pth




In [0]:
!ls {"/usr/local/lib/python3.6/dist-packages/fastai/weights/"}

In [0]:
!ls {"/content/clouderizer/fast.ai/fastai/courses/dl1/fastai/weights"}

In [0]:
!mkdir -p /usr/local/lib/python3.6/dist-packages/fastai/weights/
#!( cd /content/clouderizer/fast.ai/fastai/courses/dl1/fastai/weights && tar cf - . ) | (cd /usr/local/lib/python3.6/dist-packages/fastai/weights/ && tar xvpf - )
! cp /content/clouderizer/fast.ai/fastai/courses/dl1/fastai/weights/resnext_101_64x4d.pth  /usr/local/lib/python3.6/dist-packages/fastai/weights

In [0]:
#!mkdir -p /content/clouderizer/fast.ai/fastai/courses/dl1/fastai/weights/

In [0]:
!wget http://files.fast.ai/models/weights.tgz -d
!tar -xf /content/weights.tgz 

In [0]:
mv /content/weights/resnext_101_64x4d.pth  /usr/local/lib/python3.6/dist-packages/fastai/weights/

In [0]:
!ls {"/usr/local/lib/python3.6/dist-packages/fastai/weights/"}

## TTA

*   AttributeError: ‘bool’ object has no attribute ‘mean’" error in lesson1 Dogbreed example

In [0]:
!rm -r /usr/local/lib/python3.6/dist-packages/fastai/metrics.py
#!ls {"/usr/local/lib/python3.6/dist-packages/fastai/"}
!wget https://github.com/fastai/fastai/blob/f94fe9b08da60cf76b805ce9926a87b91b5eae24/fastai/metrics.py
#!ls {"/content/"}
!mv /content/metrics.py  /usr/local/lib/python3.6/dist-packages/fastai/
!ls {"/usr/local/lib/python3.6/dist-packages/fastai/"}

# CODE

##Pre Training

In [0]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [0]:
torch.cuda.is_available()

In [0]:
torch.backends.cudnn.enabled

In [0]:
PATH = "/content/clouderizer/fast.ai/data/"
arch = resnext101_64

In [0]:
sz=224

In [0]:
bs=16 #batch size

In [0]:
!ls {"/content/clouderizer/fast.ai/data/"}

In [0]:
#!rm -r /content/clouderizer/fast.ai/data/tmp

###Making validation set

In [0]:
label_csv = f'{PATH}train.csv'
n = len(list(open(label_csv)))-1 #number of trainig examples = # labels

In [0]:
val_idxs = get_cv_idxs(n)        

In [0]:
n

In [0]:
len(val_idxs)

In [0]:
val_idxs

## Looking to the data 

In [0]:
label_df = pd.read_csv(label_csv)
label_df.head()

In [0]:
label_df.pivot_table(index='Animal', aggfunc=len).sort_values('Image_id', ascending=False) #group it up

In [0]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, 
                       max_zoom=1.1)

data = ImageClassifierData.from_csv(PATH, 'train', 
                 f'{PATH}train.csv', test_name='test', 
                 val_idxs = val_idxs , tfms=tfms, bs=bs)

In [0]:
#You can access to training dataset by saying data.trn_ds and trn_ds contains a lot of things including file names (fnames)
fn = PATH + data.trn_ds.fnames[1]; fn 

In [0]:
img = PIL.Image.open(fn); img

In [0]:
img.size

### to know all sizes am dealing with

In [0]:
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames} # match name to size 

In [0]:
row_sz, col_sz = list(zip(*size_d.values()))

In [0]:
row_sz = np.array(row_sz);  col_sz = np.array(col_sz); 

In [0]:
row_sz[:15]

In [0]:
col_sz[:15]

In [0]:
plt.hist(row_sz);

In [0]:
plt.hist(col_sz);

In [0]:
plt.hist(row_sz[row_sz<1000])

## Training

In [0]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on,
                           max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', 
               f'{PATH}train.csv', test_name='test', num_workers=4,
               val_idxs = val_idxs ,tfms=tfms, bs=bs)

    return data if sz>300 else data.resize(340, 'tmp')

In [0]:
data = get_data(224, 64)

learn = ConvLearner.pretrained(arch, data, precompute=True)

#learn.fit(1e-2, 5)
#learn.fit(0.01,4)

###choosing learning rate

In [0]:
#choosing learning rate
lrf=learn.lr_find()

In [0]:
learn.sched.plot_lr()

In [0]:
learn.sched.plot()

### Continue Training 

In [0]:
bs=64 #batch size

In [0]:
learn.precompute = False

In [0]:
learn.fit(1e-2, 2, cycle_len=1)

In [0]:
learn.save('224_pre')
learn.load('224_pre')

In [0]:
lr = 0.01

In [0]:
lrs=np.array([1e-4,1e-3,1e-2])

In [0]:
learn.set_data(get_data(224, 16))
learn.unfreeze()
learn.fit(lrs, 2, cycle_len=1, cycle_mult=2)

In [0]:
learn.save('224_unf')
learn.load('224_unf')

In [0]:
learn.set_data(get_data(299, 64))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [0]:
learn.save('229_pre')
learn.load('229_pre')

In [0]:
learn.set_data(get_data(360, 64))

In [0]:
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

In [0]:
learn.save('360_pre')
learn.load('360_pre')

In [0]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs,y) ,metrics.log_loss(y, probs)

In [0]:
learn.set_data(get_data(420, 16))

In [0]:
learn.fit(lr, 1 )

In [0]:
learn.fit(lr, 2 )

In [0]:
learn.save('420_pre')
learn.load('420_pre')

In [0]:
learn.set_data(get_data(420, 64))
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs,y) ,metrics.log_loss(y, probs)

In [0]:
learn.fit(lr, 1)

In [0]:
learn.set_data(get_data(420, 128))
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs,y) ,metrics.log_loss(y, probs)

In [0]:
learn.save('420_pree')
learn.load('420_pree')

#After Training

##Getting data to submit

In [0]:
log_preds, y = learn.TTA(is_test=True)

In [0]:
probs = np.mean(np.exp(log_preds),0)

In [0]:
probs.shape

In [0]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [0]:
df.insert(0, 'image_id', [o[5:-4] for o in data.test_ds.fnames])

In [0]:
df.head()

In [0]:
#make csv and zip it 
SUBM = f'{PATH}sub/' 
os.makedirs(SUBM, exist_ok=True) 
df.to_csv(f'{SUBM}subm4.gz', compression='gzip', index=False)

In [0]:
FileLink(f'{SUBM}subm4.gz') # download file

## Download data to pc

In [0]:
from google.colab import files
files.download("/content/clouderizer/fast.ai/fastai/courses/dl1/data1/competitions/dog-breed-identification/sub/subm.gz")